# Association rules - 3rd laboratory

- `pandas` and [`mlxtend`](http://rasbt.github.io/mlxtend/)
- Apriori algorithm and FP-Growth 

-------
## Online Retail Data Set

In [1]:
!wget https://github.com/dbdmg/data-science-lab/raw/master/datasets/online_retail.csv -O ords.csv

--2022-04-19 09:54:28--  https://github.com/dbdmg/data-science-lab/raw/master/datasets/online_retail.csv
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/dbdmg/data-science-lab/master/datasets/online_retail.csv [following]
--2022-04-19 09:54:28--  https://raw.githubusercontent.com/dbdmg/data-science-lab/master/datasets/online_retail.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45580638 (43M) [text/plain]
Saving to: ‘ords.csv’

ords.csv            100%[===================>]  43,47M  25,9MB/s    in 1,7s    

2022-04-19 09:54:33 (25,9 MB/s) - ‘ords.csv’ saved [45580638/45580638]



In [2]:
import numpy as np
import pandas as pd

In [3]:
import csv

def read_csv(file_name:str) -> list:
    """Read and return a list of list of the input csv file

    Args:
        file_name (str): input CSV file

    Returns:
        list: list of list of the input file, one list for each line
        in the csv file
    """
    with open(file_name, "r") as f:
        lol = []
        for cols in csv.reader(f):
            lol.append(cols)
    return lol

In [4]:
ords_lol = np.array(read_csv('ords.csv'))

In [5]:
def indexes_to_delete(ary:np.array) -> list:
    """Return a list containing the indexes 
    that belong to a cancelled transaction

    Args:
        ary (np.array): input array

    Returns:
        list: output list containing indexes
    """
    indexes_to_delete = []
    for transaction_id in range(ary.shape[0] - 1):
        if ary[transaction_id,0][0] == 'C':
            indexes_to_delete.append(transaction_id)
        else:
            continue    
    return indexes_to_delete


ids_to_delete = indexes_to_delete(ords_lol)


In [6]:
filtered_ords = np.delete(ords_lol, ids_to_delete, axis=0)
ids_to_delete = None

In [7]:
indexes_to_delete(filtered_ords) # return empty list

[]

In [8]:
invoices = sorted(set(filtered_ords[:,0]))

transactions = {id:[] for id in invoices}

for t in filtered_ords[1:]:
    id = t[0]
    element = t[2]
    transactions[id].append(element)    
    

In [9]:
number_of_itemsets = len(transactions.values()) 
set_of_items = sorted(set(filtered_ords[1:,2]))
matrix = np.zeros(shape=(number_of_itemsets, len(set_of_items)))

In [10]:
arys = [] #list of arrays, 1 array for each transaction
for t in transactions.values():  
    ary = np.zeros(len(set_of_items))
    for transaction_item in t:
        id = 0
        for item in set_of_items:
            if transaction_item == item:
                ary[id] = 1
                id += 1
            else:
                id += 1
                continue
    arys.append(ary)

arys = np.array(arys)

check below to see how easy is using mlxtend

In [11]:
df = pd.DataFrame(data = arys, columns=set_of_items)

In [12]:
df.head()

,,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,I LOVE LONDON MINI RUCKSACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,...,wrongly coded 20713,wrongly coded 23343,wrongly coded-23343,wrongly marked,wrongly marked 23343,wrongly marked carton 22804,wrongly marked. 23343 in box,wrongly sold (22719) barcode,wrongly sold as sets,wrongly sold sets
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
from mlxtend.frequent_patterns import fpgrowth

In [14]:
good_sup = [0.5, 0.1, 0.05, 0.02, 0.01] # just a meme
fis = []
for support in good_sup:
    fi = fpgrowth(df=df, min_support=support) # fi = frequent itemset
    fis.append((support, fi))

In [15]:
for fi in fis:
    print(f"support: {fi[0]}, length of frequent itemsets: {len(fi[1])}")

support: 0.5, length of frequent itemsets: 0
support: 0.1, length of frequent itemsets: 1
support: 0.05, length of frequent itemsets: 23
support: 0.02, length of frequent itemsets: 303
support: 0.01, length of frequent itemsets: 1472


In [16]:
print(f'support {fis[3][0]}\n', fis[3][1].to_string())

support 0.02
       support            itemsets
0    0.102425              (3904)
1    0.020802              (1902)
2    0.021709              (1645)
3    0.065942               (244)
4    0.035214              (1738)
5    0.028507              (2026)
6    0.027192              (1076)
7    0.020349              (1283)
8    0.051348              (1825)
9    0.051031              (2743)
10   0.047632               (166)
11   0.044414               (162)
12   0.040789              (3506)
13   0.038477              (2897)
14   0.035486               (165)
15   0.033175              (3016)
16   0.032042              (2056)
17   0.027555              (3782)
18   0.052572              (2431)
19   0.049853              (3970)
20   0.044006              (3966)
21   0.024609              (3952)
22   0.033583              (2837)
23   0.029141              (1763)
24   0.070881              (2046)
25   0.059823              (2387)
26   0.055201              (1856)
27   0.053660              (1871)


The total number of frequent itemsets is 303, the most relevant ones are those made by more than 1 item

------------
That's how they do it in mlxtend

In [17]:
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
dataset = list(transactions.values())
te_ary = te.fit(dataset).transform(dataset)
df1 = pd.DataFrame(te_ary, columns=te.columns_)
df1

,,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,I LOVE LONDON MINI RUCKSACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,...,wrongly coded 20713,wrongly coded 23343,wrongly coded-23343,wrongly marked,wrongly marked 23343,wrongly marked carton 22804,wrongly marked. 23343 in box,wrongly sold (22719) barcode,wrongly sold as sets,wrongly sold sets
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22060,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
22061,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
22062,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
22063,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


---------

In [18]:
columns_list = list(df.columns)

In [19]:
columns_list[1450:1452]

['GARDENERS KNEELING PAD CUP OF TEA ', 'GARDENERS KNEELING PAD KEEP CALM ']

In [20]:
print(columns_list[3514])
print(columns_list[2463])

SPOTTY BUNTING
PARTY BUNTING


In [21]:
fi_02 = fis[3][1]
type(fi_02)
#fi_02.sort_values(by='support', inplace=True)
fi_02.sort_index(inplace=True)

In [22]:
def mask_itemsets(df:pd.DataFrame, column:str='itemsets', min_length_itemsets:int=2):
    mask = []
    for i in df['itemsets']:
        if len(i) >= 2:
            mask.append(True)
        else:
            mask.append(False)
    return mask


def extract_association_rules(columns_list:list, df:pd.DataFrame, mask:list):
    masked_df = df[mask]
    masked_df.sort_values(by='support', inplace=True)
    length = len(masked_df)
    association_rules = []
    for row in range(length):
        support = masked_df.iloc[row][0]
        items = [columns_list[i] for i in masked_df.iloc[row][1]]
        association_rules.append((support, items))
    return association_rules

In [23]:
mask_2min = mask_itemsets(fi_02,'itemsets',2)
ars = extract_association_rules(columns_list=columns_list, df=fi_02, mask=mask_2min)

/var/folders/2q/7qslh1r50qv5dxqpmmm9jcy00000gn/T/ipykernel_1757/4165551152.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  masked_df.sort_values(by='support', inplace=True)


In [24]:
ars

[(0.020122365737593472, ['LUNCH BAG  BLACK SKULL.', 'LUNCH BAG WOODLAND']),
 (0.020303648311806026, ['LUNCH BAG CARS BLUE', 'LUNCH BAG WOODLAND']),
 (0.020303648311806026,
  ['JUMBO BAG RED RETROSPOT', 'JUMBO BAG SCANDINAVIAN BLUE PAISLEY']),
 (0.020348968955359165,
  ['JUMBO BAG WOODLAND ANIMALS', 'JUMBO STORAGE BAG SUKI']),
 (0.020348968955359165, ['LUNCH BAG APPLE DESIGN', 'LUNCH BAG SUKI DESIGN ']),
 (0.020348968955359165,
  ['WHITE HANGING HEART T-LIGHT HOLDER', 'JUMBO BAG RED RETROSPOT']),
 (0.02053025152957172,
  ['PACK OF 72 RETROSPOT CAKE CASES', '60 TEATIME FAIRY CAKE CASES']),
 (0.020620892816677998,
  ['LUNCH BAG SPACEBOY DESIGN ', 'LUNCH BAG SUKI DESIGN ']),
 (0.020666213460231134,
  ['CHARLOTTE BAG SUKI DESIGN', 'WOODLAND CHARLOTTE BAG']),
 (0.021210061182868795,
  ['JUMBO BAG PINK POLKADOT', 'JUMBO SHOPPER VINTAGE RED PAISLEY']),
 (0.021300702469975074,
  ['ALARM CLOCK BAKELIKE PINK', 'ALARM CLOCK BAKELIKE RED ']),
 (0.021346023113528213, ['LUNCH BAG CARS BLUE', 'LUNCH B

In [25]:
from mlxtend.frequent_patterns import association_rules

fi_01 = fis[-1][1]

association_rules(fi_01, metric='confidence', min_threshold=.9)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(3547, 724, 723)",(2861),0.012282,0.046862,0.011104,0.904059,19.292130,0.010528,9.934635
1,"(3547, 3981, 723)",(2861),0.011919,0.046862,0.010968,0.920152,19.635547,0.010409,11.936924
2,(1731),(1732),0.010877,0.010741,0.010016,0.920833,85.730749,0.009899,12.495903
3,(1732),(1731),0.010741,0.010877,0.010016,0.932489,85.730749,0.009899,14.651385
4,"(3976, 3968)",(3975),0.011285,0.023204,0.010469,0.927711,39.980351,0.010207,13.512342
5,"(2656, 2905, 3003)",(1599),0.015001,0.046000,0.013642,0.909366,19.768622,0.012951,10.525795
6,"(2656, 3003)",(1599),0.027147,0.046000,0.024564,0.904841,19.670271,0.023315,10.025364
7,(2911),(2910),0.013823,0.017267,0.012599,0.911475,52.786627,0.012360,11.101241
8,"(2912, 2911)",(2910),0.012191,0.017267,0.011557,0.947955,54.899306,0.011346,18.882509
9,"(2910, 2911)",(2912),0.012599,0.020168,0.011557,0.917266,45.481974,0.011303,11.843191


In [26]:
from mlxtend.frequent_patterns import apriori

aprs = []
for support in good_sup:
    apr = apriori(df=df, min_support=support,)
    aprs.append((support, apr))

In [ ]:
print("fp\tapr")
for i,j in zip(fis, aprs):
    fi = i[1]
    apr = i[1]
    print(f"{len(fi)}\t{len(apr)}")

fp	apr
0	0
1	1
23	23
303	303
1472	1472


FP-growth is way more efficient (ca 11 sec vs more than 7 minutes).
Same itemsets were retrieved

In [ ]:
apr_02 = aprs[-2]
apr_01 = aprs[-1]

In [ ]:
mask = mask_itemsets(apr_02[1], min_length_itemsets=2)
apr_ars = extract_association_rules(list(df.columns), apr_02[1], mask)

/var/folders/2q/7qslh1r50qv5dxqpmmm9jcy00000gn/T/ipykernel_1470/4165551152.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  masked_df.sort_values(by='support', inplace=True)


In [ ]:
apr_ars

[(0.020122365737593472, ['LUNCH BAG  BLACK SKULL.', 'LUNCH BAG WOODLAND']),
 (0.020303648311806026, ['LUNCH BAG CARS BLUE', 'LUNCH BAG WOODLAND']),
 (0.020303648311806026,
  ['JUMBO BAG RED RETROSPOT', 'JUMBO BAG SCANDINAVIAN BLUE PAISLEY']),
 (0.020348968955359165,
  ['JUMBO BAG WOODLAND ANIMALS', 'JUMBO STORAGE BAG SUKI']),
 (0.020348968955359165,
  ['WHITE HANGING HEART T-LIGHT HOLDER', 'JUMBO BAG RED RETROSPOT']),
 (0.020348968955359165, ['LUNCH BAG APPLE DESIGN', 'LUNCH BAG SUKI DESIGN ']),
 (0.02053025152957172,
  ['PACK OF 72 RETROSPOT CAKE CASES', '60 TEATIME FAIRY CAKE CASES']),
 (0.020620892816677998,
  ['LUNCH BAG SPACEBOY DESIGN ', 'LUNCH BAG SUKI DESIGN ']),
 (0.020666213460231134,
  ['CHARLOTTE BAG SUKI DESIGN', 'WOODLAND CHARLOTTE BAG']),
 (0.021210061182868795,
  ['JUMBO BAG PINK POLKADOT', 'JUMBO SHOPPER VINTAGE RED PAISLEY']),
 (0.021300702469975074,
  ['ALARM CLOCK BAKELIKE PINK', 'ALARM CLOCK BAKELIKE RED ']),
 (0.021346023113528213, ['LUNCH BAG CARS BLUE', 'LUNCH B

-------
# COCO and Apriori

In [1]:
!wget https://raw.githubusercontent.com/dbdmg/data-science-lab/master/datasets/modified_coco.json -O coco.json

--2022-04-19 10:00:31--  https://raw.githubusercontent.com/dbdmg/data-science-lab/master/datasets/modified_coco.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 872358 (852K) [text/plain]
Saving to: ‘coco.json’

coco.json           100%[===================>] 851,91K  --.-KB/s    in 0,06s   

2022-04-19 10:00:32 (12,9 MB/s) - ‘coco.json’ saved [872358/872358]



In [6]:
import json

with open("coco.json", "r") as f:
    coco = json.load(f) # list of dictionaries. 1 dict for each image

In [7]:
type(coco)

list

**Apriori principle**: _'If an itemset is frequent, then all of its subsets must be frequent'_

Apriori algorithm implementation:
1. Scan the dataset to count the support of each item
2. Generate candidate itemsets of length k + 1 (ex. if length k is 'ab', length k + 1 will be 'abc'). 
3. Apply the Apriori principle. If a candidate itemset of length k + 1 contains at least one k-itemset that is not frequent.
4. Scan the database to count the support of k+1 itemsets and prune the ones that do not satisfy the minimum support.

In [3]:
# Toy example
toy_list = [("a","b"), 
            ("b","c","d"), 
            ("a","c","d","e"), 
            ("a","d","e"), 
            ("a","b","c"), 
            ("a","b","c","d"), 
            ("b","c"), 
            ("a","b","c"), 
            ("a","b","d"), 
            ("b","c","e")]

In [1]:
def apriori(transaction_list:list, minsup:float=0.02):
    # set of items
    itemset = list(scan(transaction_list).keys()) 
    
    # store not frequent candidates
    not_freq_candidates = [] 
    frequents = []
    
    for i in range(1, len(itemset)):
        
        # scan the transaction list to count the support
        supports = scan(transaction_list, k=i)
        supports = {k:v/len(transaction_list) for k,v in supports.items()}
        
        # prune itemsets if below minsup, return frequents and not frequents ones 
        pruned_supports, not_freq = prune(supports, minsup, return_not_freq=True)
        frequents.append(pruned_supports)
        not_freq_candidates.append(list(not_freq.keys()))
        
        # generate new candidates of length k + 1
        candidates = gen_candidates(transaction_list, k=i+1)
        
        # candidates that do not contain a subset not frequent
        pruned_candidates = [] 
        
        # check if the generated candidates contains elements not frequent
        for ca in candidates:
            ca_k_comb = combinations(ca, i-1)
            for comb in ca_k_comb:
                if comb in not_freq_candidates:
                    continue
                else:
                    pruned_candidates.append(ca)
        
        # break statement
        if i == len(itemset) - 1:
            break
        
    return frequents

from itertools import combinations

def scan(transaction_list:list, k:int=1) -> dict:
    # create a dictionary with items as keys
    # and supports as values
    d = dict()
    for transaction in transaction_list:
        # create combinations (without repetition) of size k
        combs = combinations(transaction, k)
        for item in combs:
            d[item] = d.get(item, 0) + 1
    return d


def gen_candidates(items_list:list, k:int=1):
    # generate candidates of length k
    return list(combinations(items_list, k))

def prune(k_itemsets:dict, minsup:int=1, return_not_freq:bool=False):
    # create 2 dictionaries, one containing frequent itemsets
    # while the other one has the non frequent ones
    freq = {k:v for k,v in k_itemsets.items() if v >= minsup}
    not_freq = {k:v for k,v in k_itemsets.items() if v < minsup}
    
    # optional parameter that will return frequent dict 
    # as well as the not frequent one
    if return_not_freq:
        return freq, not_freq 
    return freq


In [5]:
apriori(toy_list, .5)

[{('a',): 0.7, ('b',): 0.8, ('c',): 0.7, ('d',): 0.5},
 {('a', 'b'): 0.5, ('b', 'c'): 0.6},
 {},
 {}]

In [8]:
annot = []
for image in coco:
    annot.append(tuple(image['annotations']))


In [9]:
annot[:3]

[('car', 'car', 'train', 'stop sign'),
 ('person',
  'person',
  'person',
  'person',
  'person',
  'bench',
  'chair',
  'chair',
  'chair',
  'chair',
  'chair',
  'chair',
  'potted plant',
  'potted plant',
  'potted plant',
  'dining table',
  'dining table',
  'dining table'),
 ('stop sign',)]

In [10]:
apriori(annot, .02)

Error: Canceled future for execute_request message before replies were done